In [4]:
from ultralytics import YOLO
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO

detector = YOLO("/root/Projects/Hackathon/MacroVision/Model/best.pt")
food_classifier = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
food_classifier.eval()


def analyze_image(image_url):
    try:
        response = requests.get(image_url, timeout=10)
        img = Image.open(BytesIO(response.content))

        detections = detector.predict(img, conf=0.3, save=False)
        
        results = []
        if len(detections[0].boxes) > 0:
            for box in detections[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                crop = img.crop((x1, y1, x2, y2))
                
                food_type = classify_food(crop)
                if food_type != "non-food item":
                    results.append({
                        "bbox": [x1, y1, x2, y2],
                        "food": food_type,
                        "confidence": float(box.conf.item())
                    })

            plotted = detections[0].plot()
            Image.fromarray(plotted).show()
        else:
            print("No food items detected in the image.")
        
        return results
    except Exception as e:
        print(f"Error processing image: {str(e)}")
        return []

if __name__ == "__main__":
    test_image = "https://www.southernliving.com/thmb/7nXwDG77BEDSYw0zUbxmK6m-zpU=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/GettyImages-96776928-2000-32160ee48e3146f693ee3ee8d8b6dca5.jpg"
    
    results = analyze_image(test_image)
    
    if results:
        print("\nDetection Results:")
        for idx, item in enumerate(results, 1):
            print(f"{idx}. {item['food']} (Confidence: {item['confidence']:.2f})")
            print(f"   Bounding Box: {item['bbox']}\n")
    else:
        print("No food items found in the image.")


0: 384x640 (no detections), 169.4ms
Speed: 4.7ms preprocess, 169.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
